&nbsp;
&nbsp;

# Welcome to Feature Factory for Biodegradability

Feature factory is an online infrastructure that allows one to quickly prototype and test features for different machine learning problems. 

Before beginning to use Feature Factory, we highly recommend that you familiarize yourself with what IPython Notebook. IPython Notebook is an interactive python kernel that allows you to run code in different cells. Variables created by the code live in the IPython Notebook python kernel and can be accessed at any time, by any cell. More information can be found at http://ipython.org/notebook.html

# Creating your own IPython Notebook

To get started with Feature Factory, please clone the Template notebook. To do this, click "File"->"Make a Copy". This should spawn a new tab within your browser with the copied notebook. Rename the notebook to your liking and make all edits on that notebook.

&nbsp;
&nbsp;


# Biodegradability Machine Learning Competition

## Problem Statement

The persistence of chemicals in the environment (or to environmental inuences) is welcome only until the time the chemicals fulll their role. After that time or if they happen to be at the wrong place, the chemicals are considered pollutants.
In this phase of chemicals' life-span we wish that the chemicals disappear as soon as possible. The most ecologically acceptable (and a very cost-eective) way of 'disappearing' is degradation to components which are not considered pollutants (e.g. mineralization of organic compounds). Degradation in the environment can take several forms, from physical pathways (erosion, photolysis, etc.), through chemical pathways (hydrolysis, oxydation, diverse chemolises, etc.) to biological pathways (biolysis). Usually the pathways are combined and interrelated, thus making degradation even more complex.

In our study we focus on biodegradation in an aqueous environment under aerobic conditions, which affects the quality of surface and groundwater.

In this competition, you will be given a dataset of chemical properties measured during a study on biodegradation in an aqueous environment under aerobic conditions, in which the water/octanol partition coefficient (LOGP) value of each molecule has been used to classify them into multiple classes.

You are challenged to work with this dataset and attempt to identify and derive or generate the features which would help the most in predicting the logp class of any of the molecules based on the rest of values measured.


## Data

The dataset is in a relational format, split among mutliple files. When using **commands.get_sample_dataset()** to retrieve the dataset, the files are provided as a list of *pandas.DataFrame* objects with the following columns:

![Biodegradability Data Model](https://relational.fit.cvut.cz/assets/img/datasets-generated/Biodegradability.svg)

## Step-by-Step Example

Step 1: Import the feature factory infrastructure

In [1]:
from problems.biodegradability import commands

&nbsp;
&nbsp;

Step 2: Create a username/password or login into an existing account. If you create an account and it is successful, you don't need to login - you are logged in automatically. 

In [2]:
commands.create_user('a_user', 'a_password')

user successfully created


In [2]:
commands.login('stair_2', '')

user successfully logged in


&nbsp;
&nbsp;

Step 3: To ensure that this notebook is mapped to your username, it is required that you execute the command below. 

In [3]:
commands.add_notebook('Biodegradability_stair_2')

Notebook Biodegradability_stair_2 successfully registered


&nbsp;
&nbsp;

Step 4: Get a sample dataset. This will allow you to test your feature before running it on the full data in the server. Remember that the dataset as a list of [Pandas DataFrames](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

In [4]:
dataset = commands.get_sample_dataset()
# dataset[0] <- this refers to the molecule data
# dataset[1] <- this refers to the atom data
# dataset[2] <- this refers to the bond data
# dataset[3] <- this refers to the gmember data
# dataset[4] <- this refers to the group data

In [58]:
dataset[0][:5]

,molecule_id,activity,mweight,activitynorm,logpnorm,mweightnorm
0,i100_02_7i,9.618279,266.648116,0.000085,0.902916,0.641276
1,i100_21_0i,5.365895,246.420397,0.000076,0.545010,0.605700
2,i100_41_4i,2.612409,190.982452,0.000064,0.100432,0.090328
3,i100_42_5i,6.144340,134.751692,0.000088,0.700206,0.928509
4,i100_44_7i,8.570449,207.339516,0.000131,0.039574,0.659513


In [6]:
dataset[1][:5]

,atom_id,molecule_id,type
0,i100_02_7_10i,i94_36_0i,c
1,i100_02_7_10_1i,i50_32_8i,cl
2,i100_02_7_1i,i108_88_3i,c
3,i100_02_7_2i,i593_60_2i,o
4,i100_02_7_3i,i120_12_7i,c


In [8]:
dataset[2][:5]

,atom_id,atom_id2,type
0,i111_76_2_1i,i56_55_3_10_1i,1
1,i510_15_6_19_1i,i95_50_1_8i,7
2,i119_93_7_10i,i79_01_6_2_1i,1
3,i96_45_7_2i,i95_95_4_4_1i,7
4,i842_07_9_9i,i115_07_1_3_1i,1


In [10]:
dataset[3][:5]

,atom_id,group_id
0,i100_21_0_9i,g972
1,i72_55_9_3i,g1241
2,i1861_32_1_1_2i,g1000
3,i1634_04_4_4_1i,g1549
4,i96_12_8_4_3i,g1014


In [8]:
dataset[4][:5]

,group_id,type
0,g0,sulfo
1,g1,sulfo
2,g10,nitro
3,g100,methyl
4,g1000,c2n


In [120]:
def Biodegradability_Feature_Extractor_stair_2(dataset):
    
    import pandas as pd
    from sklearn.preprocessing import OneHotEncoder, LabelEncoder

    mar = dataset[3]
    mar = mar.merge(dataset[4], how='outer', on='group_id')
    mar = mar.merge(dataset[2], how='outer', on='atom_id')
    mar = mar.merge(dataset[1], how='outer', on='atom_id')

    le = LabelEncoder()
    #mar['type_y'] = le.fit_transform(mar['type_y'])
    mar['type_y'] = mar['type_y'].fillna(0)
    mar = mar.fillna('Nan')
    mar['atom_id'] = le.fit_transform(mar['atom_id'])
    mar['group_id'] = le.fit_transform(mar['group_id'])
    mar['type_x'] = le.fit_transform(mar['type_x'])
    mar['atom_id2'] = le.fit_transform(mar['atom_id2'])
    mar['type'] = le.fit_transform(mar['type'])
    
    enc = OneHotEncoder(sparse=False)
    mar = pd.concat([ mar['molecule_id'] , pd.DataFrame(
                enc.fit_transform(mar.drop(['molecule_id'], axis=1).as_matrix()))], axis=1)
    #X = mar
    mar = mar.groupby(['molecule_id'], as_index=False).sum()
    mar = mar.iloc[1:,:]
    features = mar.merge(dataset[0], how='inner', on='molecule_id')
    X = features.drop(['molecule_id'], axis=1)
        
    return X
    

In [121]:
#Biodegradability_Feature_Extractor_stair_2(dataset)
commands.cross_validate(Biodegradability_Feature_Extractor_stair_2)

Obtaining dataset
Extracting features
Cross validating


0.32523590296143928

&nbsp;
&nbsp;

Step 5: Define your feature extraction function.

The name you give to the function is the name which will be used later on to register your feature extaction function and the score which it obtains.

Your function should simply take in the dataset list as a parameter and output a N x M numpy matrix or pandas dataframe where N is number of molecules, one row per molecules, and M is the number of features which will be used for the prediction.
Bear in mind that sorting is important and that, in order to properly evaluate your function score, the extracted features should preserve the order of the molecules table.

Also note that, even though the system allows you to do so, any feature extraction function which makes use of the outcome column will be disqualified.

**WARNING:** Your functions have to be self contained!

This means that you can use helper functions or import external modules but that any import or variable definition needs to be made within the functions which use them.

Cross validation is (intentionally) run in a separated process in order to make sure that this scope pattern is preserved, and will fail if the function uses anything defined somewhere else in the notebook.

You might be wondering why we require this. The reason is that the code of your function might be executed and further evaluated in different environments where the variables and modules defined in your notebook will not be available.

In [3]:
def example_feature(dataset):
    return dataset[0][['mweight']]

&nbsp;
&nbsp;

Step 6: Evaluate the score of your feature extraction function before submitting it.

You can make use of the cross_validate command as many times a needed in order to have a preview of what the score of your function will be.

In [123]:
commands.cross_validate(Biodegradability_Feature_Extractor_stair_2)

Obtaining dataset
Extracting features
Cross validating


0.32523590296143928

&nbsp;
&nbsp;

Step 7: Register your function in the system

Once you are satisfied with the results, you can call the add_feature command passing your function as an argument.
This will cross_validate the function again and store your code and your score for future analysis.

Again, remember that your function code must be self contained and import or define everything it needs to be run successfully.

In [124]:
commands.add_feature(Biodegradability_Feature_Extractor_stair_2)

Obtaining dataset
Extracting features
Cross validating
Your feature Biodegradability_Feature_Extractor_stair_2 scored 0.3252359029614393
Feature Biodegradability_Feature_Extractor_stair_2 successfully registered


&nbsp;
&nbsp;

Step 8: (Optional) Modify and update your function code.

If you discover that your function can be improved you can add it again into the system as many times as required with the same function name.

However, for improved clarity, we recommend you to use this option only to fix problems or make small improvements within a similar approach.

So, in case you want start a different feature extraction strategy, we strongly recommend you to register it with a new name.

In [13]:
def imports():    # We need to import pandas within our functions
    global np
    import numpy as np

def compute_log(feature):
    """Compute log of the given column."""
    return np.log(feature)
    
def example_feature(dataset):
    imports()
    
    df = dataset[0][['mweight', 'activitynorm']].copy()
    df['log'] = compute_log(df['mweight'])
    return df

commands.add_feature(example_feature)

Obtaining dataset
Extracting features
Cross validating
Your feature example_feature scored 0.2932986300603109
Feature example_feature successfully registered
